In [1]:
import sys
from pathlib import Path

import numpy as np
import tensorflow as tf

SCRIPT_DIR = Path.cwd()
sys.path.append(str(SCRIPT_DIR.parent))

from data_utils.data_iterator import DataIterator

## Load the processed data

In [2]:
training_data = np.load(open("training_data/mushrooms_training_data.npy", 'rb'))
training_labels = np.load(open("training_data/mushrooms_training_labels.npy", 'rb'))
validation_data = np.load(open("training_data/mushrooms_validation_data.npy", 'rb'))
validation_labels = np.load(open("training_data/mushrooms_validation_labels.npy", 'rb'))
test_data = np.load(open("training_data/mushrooms_test_data.npy", 'rb'))
test_labels = np.load(open("training_data/mushrooms_test_labels.npy", 'rb'))

In [3]:
training_data.shape

(6500, 117)

In [4]:
training_labels.shape

(6500, 1)

In [5]:
validation_data.shape

(812, 117)

## Model hyperparameters and parameters

In [6]:
# Hyperparameters
epochs = 20
learning_rate = 0.001
batch_size = 512

In [7]:
# Parameters
hidden_1_units = 50
hidden_1_activation = tf.nn.relu
hidden_2_units = 20
hidden_2_activation = tf.nn.relu
hidden_3_units = 5
hidden_3_activation = tf.nn.relu

## Create model
### Placeholders

In [8]:
x = tf.placeholder(dtype=tf.float32, shape=[None, training_data.shape[1]], name="inputs")
y = tf.placeholder(dtype=tf.float32, shape=[None, 1], name="labels")

### Create hidden layers and output layer

In [9]:
fully_connected_1 = tf.layers.dense(x, units=hidden_1_units, activation=hidden_1_activation, name="hidden_1")
fully_connected_2 = tf.layers.dense(fully_connected_1, units=hidden_2_units, activation=hidden_2_activation, name="hidden_2")
fully_connected_3 = tf.layers.dense(fully_connected_2, units=hidden_3_units, activation=hidden_3_activation, name="hidden_3")
output_layer = tf.layers.dense(fully_connected_3, units=1, activation=tf.nn.relu, name="output")

### Cost and Optimizer

In [10]:
init = tf.global_variables_initializer()

with tf.variable_scope("Cost_and_Optimizer"):
    loss_op = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=output_layer, name="loss_op")
    cost = tf.reduce_sum(loss_op, name="cost")
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train = optimizer.minimize(cost)

### Create lists of batchs

In [12]:
training_iterator = DataIterator(training_data, training_labels, batch_size=batch_size, shuffle=True)

## Model training

In [17]:
tf.summary.scalar("Cost", cost)
tf.summary.histogram("fully_connected_1", fully_connected_1)
tf.summary.histogram("output_layer", output_layer)

with tf.Session() as sess:
    sess.run(init)
    merged_summary = tf.summary.merge_all()
    
    # Handle old tensorboard file with same hyperparameters
    tensorboard_job_name = "{}-{}-{}-lr_{}-e_{}-b_{}".format(hidden_1_units, hidden_2_units, hidden_3_units,
                                                             learning_rate, epochs, batch_size)
    tensorboard_log_dir = Path(Path.cwd(), "tensorboard_logs", tensorboard_job_name)
    writer = tf.summary.FileWriter("./tensorboard_logs/{}".format(tensorboard_job_name))
    if len(list(tensorboard_log_dir.iterdir())) > 0:
        for file in list(tensorboard_log_dir.iterdir()):
            file.unlink()
    
    writer.add_graph(tf.get_default_graph())
    
    training_steps = 0
    for e in range(epochs):
        
        for training_data_batch, training_labels_batch in training_iterator:
            
            if training_steps%5 == 0:
                
                output, batch_cost = sess.run([output_layer, cost], feed_dict={x: training_data_batch,
                                                                               y: training_labels_batch})
                
                predictions = output > 0.5
                accuracy = np.sum(training_labels_batch == predictions) / training_labels_batch.shape[0]
                
                validation_output, validation_cost = sess.run([output_layer, cost], feed_dict={x: validation_data,
                                                                                               y: validation_labels})
                validation_predictions = validation_output > 0.5
                validation_accuracy = np.sum(validation_labels == validation_predictions) / validation_labels.shape[0]
                
                print("Epoch N°: {} - Batch N°: {}\n"
                      "Training cost = {} - Training accuracy = {}%\n"
                      "Validation cost = {} - Validation accuracy = {}%".format(e, training_steps, batch_cost, accuracy * 100,
                                                                               validation_cost, validation_accuracy * 100))
            
            s = sess.run(merged_summary, feed_dict={x: training_data_batch, y: training_labels_batch})
            writer.add_summary(s, training_steps)
            training_steps += 1
            
            sess.run(train, feed_dict={x: training_data_batch, y: training_labels_batch})

Epoch N°: 0 - Batch N°: 0
Training cost = 354.1033935546875 - Training accuracy = 45.8984375%
Validation cost = 565.1396484375 - Validation accuracy = 81.77339901477832%
Epoch N°: 0 - Batch N°: 5
Training cost = 307.39910888671875 - Training accuracy = 76.171875%
Validation cost = 603.3128662109375 - Validation accuracy = 86.94581280788178%
Epoch N°: 0 - Batch N°: 10
Training cost = 263.9947509765625 - Training accuracy = 85.9375%
Validation cost = 533.6166381835938 - Validation accuracy = 89.90147783251231%
Epoch N°: 1 - Batch N°: 15
Training cost = 209.97283935546875 - Training accuracy = 94.53125%
Validation cost = 516.61865234375 - Validation accuracy = 95.19704433497537%
Epoch N°: 1 - Batch N°: 20
Training cost = 190.21658325195312 - Training accuracy = 95.5078125%
Validation cost = 486.14239501953125 - Validation accuracy = 98.0295566502463%
Epoch N°: 1 - Batch N°: 25
Training cost = 125.79368591308594 - Training accuracy = 97.47191011235955%
Validation cost = 490.51055908203125 

Validation cost = 478.62548828125 - Validation accuracy = 98.52216748768473%
Epoch N°: 18 - Batch N°: 240
Training cost = 159.67002868652344 - Training accuracy = 98.828125%
Validation cost = 476.62481689453125 - Validation accuracy = 99.01477832512316%
Epoch N°: 18 - Batch N°: 245
Training cost = 145.71426391601562 - Training accuracy = 99.21875%
Validation cost = 477.8223571777344 - Validation accuracy = 98.64532019704434%
Epoch N°: 19 - Batch N°: 250
Training cost = 159.539306640625 - Training accuracy = 99.4140625%
Validation cost = 478.733154296875 - Validation accuracy = 98.52216748768473%
Epoch N°: 19 - Batch N°: 255
Training cost = 143.59945678710938 - Training accuracy = 99.609375%
Validation cost = 476.5534362792969 - Validation accuracy = 99.01477832512316%


## Tensorboard

To use tensorboard open a console, go to the project folder (`"mushrooms"`) type:

```bash
tensorboard --logdir tensoboard_logs\
```

### Tensoboard results

![output_layer_histograms](output_layer_histograms.png)
![graph](graph.png)